In [11]:
!pip install portalocker
!pip install torchmetrics

In [12]:
import argparse
import logging
import time

import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torchtext.data.utils import get_tokenizer, ngrams_iterator
from torchtext.datasets import DATASETS
from torchtext.prototype.transforms import load_sp_model, PRETRAINED_SP_MODEL, SentencePieceTokenizer
from torchtext.utils import download_from_url
from torchtext.vocab import build_vocab_from_iterator
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torchtext.vocab import GloVe
from tqdm import tqdm

torch.autograd.set_detect_anomaly(True)

FILL = '_FILL_'

### Information
- torchtext repo: https://github.com/pytorch/text/tree/main/torchtext
- torchtext documentation: https://pytorch.org/text/stable/index.html

### Constants

In [13]:
DATASET = "AG_NEWS"
DATA_DIR = ".data"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EMBED_DIM = 300
LR = 1.0
BATCH_SIZE = 128
NUM_EPOCHS = 5
PADDING_VALUE = 0
PADDING_IDX = PADDING_VALUE

In [14]:
DEVICE

'cuda'

### Get the tokenizer
- Use the WordLevel tokenizer.


In [15]:
# Get basic tokenizer
basic_english_tokenizer = get_tokenizer('basic_english')

In [16]:
basic_english_tokenizer("This is some text ...")

['this', 'is', 'some', 'text', '.', '.', '.']

In [17]:
# Needed later; set this to the tokenizer above
TOKENIZER = basic_english_tokenizer

### Get the data and get the vocabulary

In [18]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield TOKENIZER(text)

In [19]:
train_iter = DATASETS[DATASET](root=DATA_DIR, split="train")
VOCAB = build_vocab_from_iterator(yield_tokens(train_iter), specials=('<pad>', '<unk>'))

# Make the default index the same as that of the unk_token.
VOCAB.set_default_index(VOCAB['<unk>'])
VOCAB_SIZE = len(VOCAB)

### Get GLOVE embeddings

In [20]:
# Get the GloVe embeddings; this will be slow
GLOVE = GloVe()#FILL

In [21]:
len(GLOVE)
#GLOVE.vectors.shape

2196017

### Helper functions

In [22]:
def text_pipeline(text):
    return VOCAB(TOKENIZER(text))

def label_pipeline(label):
    return int(label) - 1

Nice link on collate_fn and DataLoader in PyTorch: https://python.plainenglish.io/understanding-collate-fn-in-pytorch-f9d1742647d3

In [23]:
# As before, loop through the batch and transform into tensors
"""def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text) in batch:
        # Get the label from {1, 2, 3, 4} to {0, 1, 2, 3}
        FILL

        # Return a list of ints
        processed_text = FILL
        # Append to text_list
        FILL

    # Pad and make into tensors as needed
    label_list = FILL
    text_list = FILL

    return label_list.to(DEVICE), text_list.to(DEVICE)"""
def collate_batch(batch):
    label_list, text_list = [], []
    # For each label and text in a batch, transform to a tensor of labels
    # And a tensor of tokens for each word
    # You need text_pipeline here and
    for (_label, _text) in batch:
        # Get the label from {1, 2, 3, 4} to {0, 1, 2, 3}
        label_list.append(label_pipeline(_label))
        # Return a list of ints
        # This basically returns a tensor of the token ids and then puts them into text_list
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text.clone().detach())

    # Make a tensor from the label_list
    label_tensor = torch.tensor(label_list)
    # Pad the sequence list and return a tensor
    text_tensor = pad_sequence(text_list)

    return label_tensor.to(DEVICE), text_tensor.to(DEVICE)

In [24]:
def batch_test():

    x = list([(label,_) for (label, _) in train_iter])
    x = x[0:15]
    label_list, text_list = [], []
    # For each label and text in a batch, transform to a tensor of labels
    # And a tensor of tokens for each word
    # You need text_pipeline here and
    #print(x)
    for (_label, _text) in x:
        # Get the label from {1, 2, 3, 4} to {0, 1, 2, 3}
        label_list.append(label_pipeline(_label))
        # Return a list of ints
        # This basically returns a tensor of the token ids and then puts them into text_list
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)

        text_list.append(processed_text.clone().detach())

    # Make a tensor from the label_list
    label_tensor = torch.tensor(label_list)
    # Pad the sequence list and return a tensor
    text_tensor = pad_sequence(text_list)

    return label_tensor, text_tensor

batch_test()

(tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
 tensor([[  432, 15875,    59,  ..., 38488,   203,   273],
         [  426,  1073,     9,  ...,   411,  1644,  1198],
         [    2,   855,   348,  ...,   394,  1605,  8727],
         ...,
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0]]))

### Get the data

In [25]:
train_iter = DATASETS[DATASET](root=DATA_DIR, split="train")
num_class = len(set([label for (label, _) in train_iter]))
# What are the classes?
print(f"The number of classes is {num_class} ...")

The number of classes is 4 ...


In [26]:
embedding = nn.Embedding(
            len(VOCAB),
            EMBED_DIM,
            padding_idx=PADDING_IDX
        )
embedding.to(DEVICE)

Embedding(95812, 300, padding_idx=0)

### Set up the model

Good reference on this type of model
- CNN for Sentence Classification: https://arxiv.org/pdf/1408.5882.pdf

You can build CNN models with either CNN1d or CNN2d.

In [27]:
class CNN1dTextClassificationModel(nn.Module):
    def __init__(
        self,
        vocab_size,
        num_class,
        embed_dim = 300,
        use_pretrained = True,
        fine_tune_embeddings = True,
        debug = False
    ):

        super(CNN1dTextClassificationModel, self).__init__()

        self.embedding = nn.Embedding(
            vocab_size,
            embed_dim,
            padding_idx=PADDING_IDX
        )
        self.embedding.to(DEVICE)

        if use_pretrained:
            # Set the embeddings to not requiring gradients since we'll try and modify
            self.embedding.weight.requires_grad = False
            for i in range(vocab_size):
                # Get the token for the index i
                token = VOCAB.lookup_token(i)
                # Modify the embedding for index i by the embedding for that token ; How exacty do we modify this emebedding ?
                # Do this only if token is in the stoi dictionary for GLOVE
                if token in GLOVE.stoi:
                  self.embedding.weight[i,:] = GLOVE.get_vecs_by_tokens(TOKENIZER(token))

            self.embedding.weight.requires_grad = True#FILL
        else:
            # Otherwise, initialize the weights as specified below
            self.init_weights()

        # If weights do not get changed, turn off gradients for the GloVe embeddings
        if not fine_tune_embeddings:
            self.embedding.weight.requires_grad = False

        # Define a Conv1d layer that collapses all the channels and does not collapse the time dimension ; does the imply that dilation remains at size 1?
        self.cnn1 = nn.Conv1d(in_channels=EMBED_DIM, out_channels=1, kernel_size=1, stride=1)

        # Define 3 Conv1d layers each having 1 filter and kernel sizes 2, 3 and 4
        self.cnn2 = nn.Conv1d(in_channels=EMBED_DIM, out_channels=1, kernel_size=2, stride=1)
        self.cnn3 = nn.Conv1d(in_channels=EMBED_DIM, out_channels=1, kernel_size=3, stride=1)
        self.cnn4 = nn.Conv1d(in_channels=EMBED_DIM, out_channels=1, kernel_size=4, stride=1)  # figure out in_channels like depending on the shape we expect to get in teh forward algorithm
        """
        self.cnn2 = nn.Conv1d(in_channels=1 out_channels=1, kernel_size=2, stride=1)
        self.cnn3 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=1)
        self.cnn4 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=4, stride=1)

        """

        # A linear map from some dimensions to num_class (you need to figure it out)

        self.fc = nn.Linear(3, num_class, device = DEVICE) #check
        #torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)
        self.debug = debug

        """"Why dont we have a self.maxpooling1 set to nn.Maxpool1d(kernel_size= <somesize>)"""

        self.maxpool1 = nn.MaxPool1d(kernel_size=1, stride=1)
        self.maxpool2 = nn.MaxPool1d(kernel_size=2,stride=1)
        self.maxpool3 = nn.MaxPool1d(kernel_size=3,stride=1)
        self.maxpool4 = nn.MaxPool1d(kernel_size=4,stride=1)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.relu3 = nn.ReLU()


    def init_weights(self):
        initrange = 0.5
        # Initialize the embedding weight matrix to uniform between the [-0.5, 0.5]
        self.weight.data.uniform_(-initrange, initrange)
        # Initialize the weight matrix of fc to uniform between the [-0.5, 0.5]
        self.fc.weight.data.uniform_(-initrange, initrange)
        # Initialize the bias for fc to zero
        self.fc.bias.data.zero_()

    # B = batch_size, L = sequence length, D = vector dimension
    def forward(self, text):

        # B X L X D
        # Get the embeddings for the text passed in
        embedded = self.embedding(text)

        if self.debug:
            print('embedding', embedded.shape)
        #print(embedded.shape)
        # B X D X L
        # Transpose the embedding above as needed
        embedded = torch.transpose(embedded,0,1)
        #print(embedded.shape)
        embedded = torch.transpose(embedded,1,2)
        #print(embedded.shape)
        # B X 1 X L
        # Pass through cnn1
        cnn1 = self.cnn1(embedded)
        if self.debug:
            print('cnn1', cnn1.shape)

        # B X 1
        # Take Max pooling over time
        #max1 = self.maxpool1(cnn1[:,:]) drop a dimension why would we do this how does it help us get a prediction
        cnn1 = self.maxpool1(cnn1)
        if self.debug:
          print('cnn1 after max pool', cnn1.shape)
        #print('cnn1 after max pool', cnn1.shape)
        # B X 1 X L - 1
        # Pass through cnn2 and add a RELU
        cnn2 = self.cnn2(embedded)
        if self.debug:
            print('cnn2', cnn2.shape) # define relu1
        cnn2 = self.relu1(cnn2)
        if self.debug:
            print('cnn2', cnn2.shape)
        # B X 1 X L - 2
        # Pass through cnn3 and add a RELU
        cnn3 = self.cnn3(embedded)
        if self.debug:
            print('cnn3', cnn3.shape) #define relu 2
        cnn3 = self.relu2(cnn3)
        if self.debug:
            print('cnn3', cnn3.shape)
        # B X 1 X L - 3
        # Pass through cnn4 and add a RELU
        cnn4 = self.cnn4(embedded)#FILL
        if self.debug:
            print('cnn4', cnn4.shape) #define relu 3
        cnn4 = self.relu3(cnn4)
        if self.debug:
            print('cnn4', cnn4.shape)

        # B X 1 in all cases
        # Apply max pooling to each of cnn2, cnn3 and cnn4 #define maxpooling for cnn2, cnn3, and cnn4
        cnn2 = self.maxpool2(cnn2) #FILL
        cnn3 = self.maxpool3(cnn3)#FILL
        cnn4 = self.maxpool4(cnn4) #FILL

        # B X 1 in all cases
        # Apply max pooling over time
        if self.debug:
            print('cnn2 after max', cnn2.shape)

        # Add to each cnn2, 3, 4 a skip connection to cnn1 and average the results
        #What is a skip connection
        cnn2 = (cnn2 + cnn1[:, :, :-2]).mean(axis=2)#FILL The main question is why we are doing this skip step
        cnn3 = (cnn3 + + cnn1[:, :, :-4]).mean(axis=2)#FILL
        cnn4 = (cnn4 + cnn1[:, :, :-6]).mean(axis=2)#FILL lets start by reviewing what exactly skip connection
        if self.debug:
            print('cnn2 after skip connection', cnn2.shape)
        # B X 3
        # Concatenate the above
        cnn_concat = torch.concat((cnn2, cnn3, cnn4), 1) #FILL #concat the maxpooling over [cnn2, cnn3, cnn4 ] apparently they are 3 column vectors with 128 entries or BATCH_SIZE entries 128 x 3
        if self.debug:
            print('cnn concat', cnn_concat.shape)
            # Set the debug to False after the first forward pass
            self.debug = False

        # Pass through an appropriate Linear layer to get the right dimensions needed
        out = self.fc(cnn_concat)#FILL
        #I think it means passing it through self,fc

        return out

### Set up the model

In [28]:
# If this is True, we will initialize the Embedding layer with GLOVE
USE_PRETRAINED = True,

# If this is True, we will allow for gradient updates on the nn.Embedding layer
FINE_TUNE_EMBEDDINGS = True

# Set the loss appropriately
criterion = nn.CrossEntropyLoss()

model = CNN1dTextClassificationModel(len(VOCAB), num_class,
        embed_dim = 300,
        use_pretrained = USE_PRETRAINED,
        fine_tune_embeddings = FINE_TUNE_EMBEDDINGS,
        debug = False).to(DEVICE)

# Set the optimizer to SGD
# Add an L2 regularizer of 0.00001
optimizer = torch.optim.SGD(model.parameters(),lr=LR)


# Set the scheduler to StepLR with gamma=0.1 and step_size = 1.0
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1.0, gamma = 0.1)
# study more deeply what the scheduler actually does for optimization steps

### Set up the data

In [29]:
train_iter, test_iter = DATASETS[DATASET]()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])


train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch
)

### Train the model

In [30]:
def train(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_acc, total_count = 0, 0
    total_loss, total_batches = 0.0, 0
    total_zero_gradients_percentage = []
    log_interval = 10

    for idx, (label, text) in tqdm(enumerate(dataloader)):
        # Set gradients to zero
        optimizer.zero_grad()

        # Get the predictions
        predicted_label = model(text)

        # Get the loss
        loss = criterion(predicted_label, label) #what loss is used for this #FILL

        # Do back propagation and get the gradients
        #FILL
        loss.backward()

        # Get the loss per batch and the number of batches
        with torch.no_grad():
            total_loss += loss.item()#FILL
            total_batches += 1#FILL

        # Loop through all the parameters
        # Specifically, for this batch, get the percentage of zero gradients across all parameters
        # Append this to the list above which will print out the total every 10 batches
        total_nonzero_gradients = 0.0
        total_param_count = 0
        #print #FILL
        for x in model.parameters():
          if x.requires_grad:
            total_nonzero_gradients += torch.count_nonzero(x.grad)
            total_param_count += x.grad.numel()
        # Append to total_zero_gradients_percentage
        total_zero_gradients_percentage.append(((total_param_count-total_nonzero_gradients)/total_param_count))#FILL


        # Clip the gradient at ?? Should we use 10.0 or 0.1 with the learning rate we picked and the default notebook setting?
        # Use the above loop to help you figure this out
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)#FILL

        # Do an optimization step
        optimizer.step()#FILL

        # Get the accuracy
        total_acc += (predicted_label.argmax(1) == label).sum().item()#FILL
        total_count += label.size(0) #FILL

        # Log results
        if idx % log_interval == 0 and idx > 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f} "
                "| loss {:8.3f} "
                "| zero gradients percentage {:8.3f}".format(
                    epoch, idx,
                    len(dataloader),
                    total_acc / total_count,
                    total_loss / total_batches,
                    torch.tensor(total_zero_gradients_percentage).mean().item()
                    )
            )
            # Reset variables as needed
            total_acc, total_count = 0, 0
            total_loss, total_batches = 0.0, 0
            total_zero_gradients = []

In [31]:
def evaluate(dataloader, model, criterion):
    model.eval()
    total_acc, total_count = 0, 0
    total_loss = 0.0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(input=predicted_label, target=label)
            total_acc += (predicted_label.argmax(1) == label).sum().item() #FILL
            total_count +=  label.size(0) #FILL
            total_loss += loss.item()#FILL
    return total_acc / total_count, total_loss / total_count

In [33]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, model, optimizer, criterion, epoch)
    accu_val, loss_val = evaluate(valid_dataloader, model, criterion)
    scheduler.step()
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch,
            time.time() - epoch_start_time,
            accu_val,
            loss_val
            )
    )
    print("-" * 59)

print("Checking the results of test dataset.")
accu_test, loss_test = evaluate(test_dataloader, model, criterion)
print("test accuracy {:8.3f}".format(accu_test))
print("test loss {:8.3f}".format(loss_test))

13it [00:00, 19.60it/s]

| epoch   1 |    10/  891 batches | accuracy    0.884 | loss    0.355 | zero gradients percentage    0.978


25it [00:01, 19.66it/s]

| epoch   1 |    20/  891 batches | accuracy    0.906 | loss    0.312 | zero gradients percentage    0.978


33it [00:01, 20.14it/s]

...

| epoch   5 |   870/  891 batches | accuracy    0.890 | loss    0.379 | zero gradients percentage    0.978


883it [00:52, 19.40it/s]

| epoch   5 |   880/  891 batches | accuracy    0.887 | loss    0.332 | zero gradients percentage    0.978


891it [00:53, 16.72it/s]


| epoch   5 |   890/  891 batches | accuracy    0.885 | loss    0.381 | zero gradients percentage    0.978
-----------------------------------------------------------
| end of epoch   5 | time: 53.94s | valid accuracy    0.892 
-----------------------------------------------------------
Checking the results of test dataset.
test accuracy    0.889
test loss    0.003
